In [4]:
import re
import requests
import json
import time
import pandas as pd
import jsonpath

headers = {
    "authority": "api.bilibili.com",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36",
    "accept": "application/json, text/plain, */*",
}


def bvid_url2aid(url):
    url = url.strip("/")
    s_pos = url.rfind("/") + 1
    r_pos = url.rfind("?")
    if r_pos == -1:
        bvid = url[s_pos:]
    else:
        bvid = url[s_pos:r_pos]
    api_url = f"https://api.bilibili.com/x/web-interface/view?bvid={bvid}"
    print(api_url)
    res = requests.get(api_url, headers=headers)
    res.encoding = "u8"
    data = res.json()['data']
    return data["title"], data["aid"]


def bilibili_comment(oid, n=1, all_comment=False):
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36"}

    t = str(int(time.time()*1000))
    url = f"https://api.bilibili.com/x/v2/reply/main?next={3}&type=1&oid={oid}&mode=3&plat=1&_={t}"
    res = requests.get(url, headers=headers)
    data = res.json()
    replies = jsonpath.jsonpath(
        data, "$..replies[*]" if all_comment else "$.data.replies[*]")
    names = jsonpath.jsonpath(replies, "$[*].member.uname")
    messages = jsonpath.jsonpath(replies, "$[*].content.message")
    times = jsonpath.jsonpath(replies, "$[*].ctime")
    likes = jsonpath.jsonpath(replies, "$[*].like")
    df = pd.DataFrame({"昵称": names, "评论": messages, "时间": times, "点赞数": likes})
    df.时间 = pd.to_datetime(df.时间, unit='s')
    return df


url = "https://www.bilibili.com/video/BV1AY4y1v7Xe"
title, aid = bvid_url2aid(url)
print(title, aid)
comment1 = bilibili_comment(aid, all_comment=True)
# comment1
dfs = []
for i in range(1, 11):
    dfs.append(bilibili_comment(aid, i))
df = pd.concat(dfs)
df

https://api.bilibili.com/x/web-interface/view?bvid=BV1AY4y1v7Xe
尚大毕设项目，轻松搞定计算机毕业设计 638237968


,昵称,评论,时间,点赞数
0,TRADER2,资料在哪?公众号那个百度网盘链接根本不能下载!![大哭],2022-06-05 04:02:25,3
1,满心-欢囍,答辩已过，感谢尚硅谷,2022-06-02 10:39:47,2
2,时光老头真好啊,p15报错，密码正确也登不上去\nerrError: Request failed with...,2022-05-27 15:53:13,2
3,白发与仙,p18\n前端表单参数传递为json字符串，后端接收需要添加@responseBody注解使...,2022-05-11 07:15:23,4
4,璃茉酱vv,上传头像出现问题怎么改啊？,2022-04-28 16:19:23,2
...,...,...,...,...
15,sjijizf,源码在哪下载啊还有资料在哪,2022-04-13 08:14:25,5
16,你脸红什么啊啊啊啊啊,哪位好心人发下sql文件,2022-04-13 08:04:47,2
17,过龍门,这也太好了，跟我毕设契合度太高了,2022-04-13 06:59:31,2
18,调皮D原始人,甚至教了论文怎么写，PPT怎么做[泼水节],2022-04-13 06:56:37,2
